In [181]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [182]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'euw1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344

In [183]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [184]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,OcVNgTWf7sO6zRy6-QL2_16cRb3xqkjaVgKelQEYftY6mh...,b7z8JF9hKX-AbY-w_tzuBXO_0bqUdiTqh2gI4DqUveNf-9...,D-joNrgVY5D1HQjhdzC4ToHDMadctI8V772jODtt5LUCrr...,SLY L3SCoco,907,1656427876396,2,OcVNgTWf7sO6zRy6-QL2_16cRb3xqkjaVgKelQEYftY6mh...,SLY L3SCoco,694,I,61,32,False,False,True,False
1,nCZoLVHLgRd5Uxrjy6KG5LUOm4-CGeecs6NwY5BsImAdOaw,omnfI3WE5g0dlD23hTVe-DSIENKojAN-alG8WfJURC6wkA,6V0CDUZfVo2vAkHU1gTyn3mkpBtlafr194J1lPzWOOqQWD...,Sologesang,1640,1656452432000,113,nCZoLVHLgRd5Uxrjy6KG5LUOm4-CGeecs6NwY5BsImAdOaw,Sologesang,1399,I,179,104,True,False,False,True
2,xCVgzEbJE9h-CgBFd9ETzkv0X4PfWrmamXdBLznHh8kULbs,vKdJsUqw3ir25_zf9jk8RdWfbHR76XQpex2UnFd-h5yhAw,0aHo0YKZgeTlSnSxObFUWss_mRUoktWaoqhaElFB8RNEDO...,Cilyo,19,1656488957000,160,xCVgzEbJE9h-CgBFd9ETzkv0X4PfWrmamXdBLznHh8kULbs,Cilyo,774,I,103,55,False,False,False,True
3,HoVV4k9rNKapN22slGfTOZmjKsA-9fkZus-D4yx2QIoKMfw,m0se6fucovRqtVdGCCyw7WFDyIXYmRjm6vUJ1j5O0tFVHbM,6ybd-VomjU8w7_MZPYhOj3t-lHA12Czp7YKobZ7RIVKqJ7...,humishvperugnota,981,1656449969000,186,HoVV4k9rNKapN22slGfTOZmjKsA-9fkZus-D4yx2QIoKMfw,humishvperugnota,747,I,122,89,False,False,True,True
4,OjFcOEaR9Rxu3ad4Ho43Deoxa5VeXmNP-5ABSH3wW_nb_XY,lNpVFjq_8XMtb16GKT8NqHOXkB9Ketzqu-WC5lkzXdutYA,ZUlf909Yrtc0M7pIZoVhVT-5MUOGSTMcNzEvIgkMJw3M5r...,BGN Kenobi,538,1656494946617,373,OjFcOEaR9Rxu3ad4Ho43Deoxa5VeXmNP-5ABSH3wW_nb_XY,BGN Kenobi,712,I,148,103,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,J-oPuYIT732izzbKpAGbS70mp82bzGXj-0NV0cQblSr51dU,6qk2792KnnM84F3GkM8vCrKzUPP3mpWPsiwa6oG0jlw8UA,yfTEY5ICE_Sod3he_6-KrtISbDEfH8pHujybiXu05DNIrX...,Noerum,3369,1656493618000,90,J-oPuYIT732izzbKpAGbS70mp82bzGXj-0NV0cQblSr51dU,Noerum,576,I,97,64,False,False,True,False
118,OaGBey17esKLBn7OCOEdt17eSDJZKqJ9mWnYUxa1VPEHtSU,MalNGZB9CqVcu6UdYdMhSztFML5gM94shpUF6KgAdA98Kw,LDs8qiU6eTGjNj8k9yPIz6f3lZFB0UNG0LCjerEEnZI_EU...,USM Yobidashi,20,1656482225000,74,OaGBey17esKLBn7OCOEdt17eSDJZKqJ9mWnYUxa1VPEHtSU,USM Yobidashi,523,I,93,56,False,False,True,False
119,EEeFY5ibd-24H9i6WoDIVPKjLTAwcnKI78p_5OM6wzNIZx4,bXF3FmjnfsjTxsIBG4FEC_z-XRczVHbHenRIqvQmft-PRj4,-ZNUkOtpvc---B9gTLmWgnCUbsm6kSSNnv6qy3_GvZPaWA...,5454XD,3836,1656343750000,353,EEeFY5ibd-24H9i6WoDIVPKjLTAwcnKI78p_5OM6wzNIZx4,5454XD,694,I,104,62,False,False,True,False
120,PLB-vvV1do2pcuF28Eq-ivfLiNzvzUpHJN0lGIX9achlDvc,UWwmobjd0zimi5PjZxJ82YdT-Zp42BB-3eAJuvyFNsDJeS8,I10uAXQJh4pA3WOTP18hC0gAhjJw4NIMoK1eQ1TehP5bhw...,PSR Cascendre,4498,1656451246000,279,PLB-vvV1do2pcuF28Eq-ivfLiNzvzUpHJN0lGIX9achlDvc,PSR Cascendre,599,I,92,45,False,False,True,False


In [185]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [186]:
len(uniq_matches_id)

1991

In [187]:
len(matches_asset)

2945

In [188]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [189]:
len(uniq_matches)

1991

In [190]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [191]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [192]:
len(latest_matches)

1441

In [193]:
# latest_matches[0]['info']

In [194]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [195]:
matches_array[0]
# len(matches_array)

{'match_id': 'EUW1_5944909932',
 'placement': 2,
 'augment0': 'TFT6_Augment_TrueTwos',
 'augment1': 'TFT6_Augment_CyberneticImplants2',
 'augment2': 'TFT6_Augment_SecondWind2',
 'Set7_Assassin': 1,
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 2,
 'Set7_Dragonmancer': 0,
 'Set7_Guild': 3,
 'Set7_Legend': 0,
 'Set7_Mirage': 0,
 'Set7_Ragewing': 1,
 'Set7_Swiftshot': 1,
 'Set7_Tempest': 1,
 'Set7_Warrior': 1,
 'TFT7_Sejuani': 2,
 'TFT7_Qiyana': 2,
 'TFT7_Shen': 2,
 'TFT7_Shen_item0': 'TFT_Item_ChainVest',
 'TFT7_Shen_item1': 'TFT_Item_Shroud',
 'TFT7_Twitch': 2,
 'TFT7_Twitch_item0': 'TFT_Item_MadredsBloodrazor',
 'TFT7_Hecarim': 2,
 'TFT7_Talon': 2,
 'TFT7_Talon_item0': 'TFT_Item_RedBuff',
 'TFT7_Talon_item1': 'TFT_Item_InfinityEdge',
 'TFT7_Talon_item2': 'TFT7_Item_CavalierEmblemItem',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'TFT_Item_GuardianAngel',
 'TFT7_Xayah_item1': 'TFT_Item_InfinityEdge',
 'TFT7_Xayah_item2': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Ornn': 2,
 'TFT7_Ornn_item0': 'TFT_Item_Gi

In [196]:
matches_league_df = pd.json_normalize(matches_array)

In [197]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Cavalier,Set7_Dragonmancer,Set7_Guild,...,TFT7_Vladimir_item2,TFT7_Skarner_item0,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Kayn_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Heimerdinger_item2
0,EUW1_5944909932,2,TFT6_Augment_TrueTwos,TFT6_Augment_CyberneticImplants2,TFT6_Augment_SecondWind2,1.0,1.0,2.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5944909932,1,TFT6_Augment_PortableForge,TFT6_Augment_SalvageBin,TFT6_Augment_TargetDummies,NaN,0.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5944909932,3,TFT7_Augment_AssassinEmblem,TFT6_Augment_TradeSectorPlus,TFT7_Augment_ScalescornEmblem,3.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5944909932,4,TFT7_Augment_SwiftshotPressTheAttack,TFT6_Augment_RichGetRicherPlus,TFT7_Augment_BestFriends2,1.0,1.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5944909932,7,TFT7_Augment_PandorasBench,TFT7_Augment_MikaelsGift,TFT6_Augment_SalvageBin,2.0,1.0,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11523,EUW1_5944511063,1,TFT6_Augment_PortableForge,TFT7_Augment_AxiomArc2,TFT6_Augment_FuturePeepers,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11524,EUW1_5944511063,3,TFT6_Augment_TrueTwos,TFT6_Augment_PortableForge,TFT7_Augment_GuardianEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11525,EUW1_5944511063,7,TFT6_Augment_TargetDummies,TFT6_Augment_LudensEcho2,TFT6_Augment_PortableForge,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11526,EUW1_5944511063,2,TFT6_Augment_ClearMind,TFT6_Augment_ComponentGrabBag,TFT7_Augment_AssassinCutthroat,2.0,1.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bruiser,Set7_Cavalier,Set7_Dragonmancer,Set7_Guild,...,TFT7_Vladimir_item2,TFT7_Skarner_item0,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Kayn_item2,TFT7_Aatrox_item1,TFT7_Aatrox_item2,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Heimerdinger_item2
0,EUW1_5944909932,2,TFT6_Augment_TrueTwos,TFT6_Augment_CyberneticImplants2,TFT6_Augment_SecondWind2,1.0,1.0,2.0,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5944909932,1,TFT6_Augment_PortableForge,TFT6_Augment_SalvageBin,TFT6_Augment_TargetDummies,NaN,0.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5944909932,3,TFT7_Augment_AssassinEmblem,TFT6_Augment_TradeSectorPlus,TFT7_Augment_ScalescornEmblem,3.0,1.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5944909932,4,TFT7_Augment_SwiftshotPressTheAttack,TFT6_Augment_RichGetRicherPlus,TFT7_Augment_BestFriends2,1.0,1.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5944909932,7,TFT7_Augment_PandorasBench,TFT7_Augment_MikaelsGift,TFT6_Augment_SalvageBin,2.0,1.0,NaN,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11523,EUW1_5944511063,1,TFT6_Augment_PortableForge,TFT7_Augment_AxiomArc2,TFT6_Augment_FuturePeepers,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11524,EUW1_5944511063,3,TFT6_Augment_TrueTwos,TFT6_Augment_PortableForge,TFT7_Augment_GuardianEmblem,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11525,EUW1_5944511063,7,TFT6_Augment_TargetDummies,TFT6_Augment_LudensEcho2,TFT6_Augment_PortableForge,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11526,EUW1_5944511063,2,TFT6_Augment_ClearMind,TFT6_Augment_ComponentGrabBag,TFT7_Augment_AssassinCutthroat,2.0,1.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)